# Detekce obličeje pomoci termokamery

Cvičení je zaměřené na práci s [termokamerama](https://en.wikipedia.org/wiki/Thermographic_camera). Termokamera je zařízení, které snímá obráz v IR spektru, podobně jako RGB kamery, které snímají ve viditelném spektru. Výstupem termokamery je `Radiometric image`, ve kterém v každém pixelu je zakódována hodnota nasnímané teploty v proprietarním formátu. Dneska pokročilejší software dovoluje exportovat nasnímané obraky ve formátu `csv`, ve kterém jsou umístěné skutečně hodnoty teplot ve škale pro nás zname.


Termokamery jsou aktivně používané v průmyslu, napr. pro kontrolu zahřívání elektickych součástek, nebo kotrolu vadností solárního panelů pomoci dronu.

![](images/solar.png)

### Import knihoven a konfigurace

In [ ]:
%run ../svz.ipynb
from collections import OrderedDict

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

<a id='new_functions'>Funkce z predchoziho cviceni</a> 

**Pozor že tady fukce jako parametr berou konturu!** 

Třída pro práci s popisnými charakteristikami tvarů a jednotlivé dílčí funkce.

In [ ]:
# Dimensionless descriptors
class ShapeDescriptors:
    def form_factor(area, perimeter):
        return (4 * np.pi * area) / (perimeter * perimeter)
    
    def roundness(area, max_diameter):
        return (4 * area) / (np.pi * max_diameter * max_diameter)
    
    def aspect_ratio(min_diameter, max_diameter):
        return min_diameter / max_diameter;
    
    def convexity(perimeter, convex_perimeter):
        return convex_perimeter / perimeter
    
    def solidity(area, convex_area):
        return area / convex_area
    
    def compactness(area, max_diameter):
        return np.sqrt(4 / np.pi * area) / max_diameter;
        
    def extent(area, bounding_rectangle_area):
        return area / bounding_rectangle_area;

# Špičatost
def form_factor(cont):
    return ShapeDescriptors.form_factor(cv2.contourArea(cont), cv2.arcLength(cont, True))

# Kulatost
def roundness(cont):
    area = cv2.contourArea(cont)
    _,radius = cv2.minEnclosingCircle(cont)
    r = ShapeDescriptors.roundness(area, 2*radius)
    if r > 1: r = 1
    return r

# Poměr stran
def aspect_ratio(cont):
    dims = cv2.minAreaRect(cont)[1]
    min_diameter = min(dims)
    max_diameter = max(dims)
    return ShapeDescriptors.aspect_ratio(min_diameter, max_diameter)
    
# Konvexita, vypouklost
def convexity(cont):
    hull = cv2.convexHull(cont, None, True, True)
    per = cv2.arcLength(cont, True)
    conv_per = cv2.arcLength(hull, True)
    r = ShapeDescriptors.convexity(per, conv_per)
    if r > 1: r = 1
    return r 

# Plnost, celistvost
def solidity(cont):
    hull = cv2.convexHull(cont, None, True, True)
    area = cv2.contourArea(cont)
    conv_area = cv2.contourArea(hull)
    r = ShapeDescriptors.solidity(area, conv_area)
    if r > 1: r = 1
    return r 
    
# Kompaktnost, hutnost
def compactness(cont):
    area = cv2.contourArea(cont)
    max_diameter = max(cv2.minAreaRect(cont)[1])
    r = ShapeDescriptors.compactness(area, max_diameter)
    if r > 1: r = 1
    return r 
    
# Dosah, rozměrnost
def extent(cont):
    area = cv2.contourArea(cont)
    w, h = cv2.minAreaRect(cont)[1]
    return ShapeDescriptors.extent(area, w*h)

#### Normalizace obrázku a převedení do BGR

In [ ]:
def normalized_image(img):
    scaled = ((img - img.min()) * (1/(img.max() - img.min()) * 255)).astype('uint8')
    return cv2.cvtColor(scaled, cv2.COLOR_GRAY2BGR)

---

Seznam předchozích funkcí pro přehlednost:
- [`connect_camera(...)`](../svz.ipynb#connect_functions)


- [`show_images(...)`](../svz.ipynb#show_functions)
- [`plot_images(...)`](../svz.ipynb#show_functions)


- [`to_gray(...)`](../svz.ipynb#preprocessing_functions)
- [`to_hsv(...)`](../svz.ipynb#preprocessing_functions)
- [`negative(...)`](../svz.ipynb#preprocessing_functions)
- [`crop(...)`](../svz.ipynb#preprocessing_functions)


- [`segmentation_one_threshold(...)`](../svz.ipynb#segmentation_functions)
- [`segmentation_auto_threshold(...)`](../svz.ipynb#segmentation_functions)
- [`segmentation_two_thresholds(...)`](../svz.ipynb#segmentation_functions)
- [`contours(...)`](../svz.ipynb#segmentation_functions)


- [`filtration_median(...)`](../svz.ipynb#filtration_functions)
- [`fill_holes(...)`](../svz.ipynb#filtration_functions)


- [`ocr(...)`](../svz.ipynb#ocr)


- [`to_intensity(...)`](../svz.ipynb#others)


- [`warp_to_cartesian(...)`](../svz.ipynb#cart_polar_functions)
- [`warp_to_polar(...)`](../svz.ipynb#cart_polar_functions)
- [`rotate(...)`](../svz.ipynb#cart_polar_functions)


- [`artificial_circle_image(...)`](../svz.ipynb#artificial_funtions)

### Úkol
Detekujte obličej na termosnimku, použijte znalosti klasifikace pomoci tvarových chrakteristik z předchozího cvičení.

#### 1) Získejte termosnimek s obličejem
Na vaším počítači je k dispozici program `Core player` pomoci kterého se dá získat termosnimek. Potřebujeme data s teplotními hodnotami ve formátu `csv`

Pro zjednodušení zobrazování lze požít `normalized_image` pomocnou funkci, která převede csv obrázek na BGR obrázek.

In [ ]:
filename = '' ###
img_csv = np.genfromtxt(filename, delimiter=';')[:, :-1]

show_images(normalized_image(img_csv))

#### 2) Segmentace obličeje
Vymyslíte způsob jak se dá segmentovat obličej v termosnimku. Zkuste experimentovat s těmto parametrem, třeba na více snimkech s vámi a vaším kolegou.

In [ ]:
_, bin_mask = ###

In [ ]:
# convert binary mask back to image format
bin_mask[bin_mask > 0] = 255
bin_mask = bin_mask.astype(np.uint8)
show_images(bin_mask)

#### 3) Získejte vlasnosti nasegmentovaných objektu
Získejte kontury segmentovaných objektů, a odfiltrujte šum, napr. pomoci plochy pod konturou, dejte pozor na to že obličej na různých snimkech může mít různé rozměry.
U získaných kontur spočtěte popisné tvarové charakteristiky, použijte znalosti z minulého cvičení.

In [ ]:
_, _, conts = ###
no_noise_conts = ###
contour_drawn = cv2.drawContours ###
show_images(contour_drawn)

In [ ]:
# tvarové charakteristiky



#### 4) Vyberte ručně vyznamne vlasnosti a jejich hodnoty na odlišení obličeje od ostatních objektů

In [ ]:
filtered_conts = ###

#### 5) Udělejte vizualizace výsledné detekce
Použijte aproximace obdélníkem nebo elipsou pro demonstrační účely

[Priklad](https://docs.opencv.org/3.1.0/dd/d49/tutorial_py_contour_features.html)

In [ ]:
img = normalized_image(img_csv)
for cont in filtered_conts:
    ###
show_images(img)###

#### 6) Vyzkoušejte vás detekční algoritmus na referenčním obrázku

In [ ]:
filenames = ['images/1.csv', 'images/2.csv', 'images/3.csv', 'images/4.csv']

###